In [1]:
!apt-get install tesseract-ocr -y
!pip install pytesseract opencv-python numpy pandas nltk openai

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 30 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,661 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 126213 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving testing foof label.jpg to testing foof label.jpg


In [ ]:
image ="food label .jpg"

In [ ]:
import cv2
import pytesseract
import numpy as np
from PIL import Image


def extract_text(image_path):
    """Extract text from an image using Tesseract OCR."""
    image = cv2.imread(image_path)

    if image is None:
        raise FileNotFoundError("Error: Image not found!")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


    gray = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]


    extracted_text = pytesseract.image_to_string(gray)
    return extracted_text.strip()


image_path = image
text = extract_text(image_path)
print("Extracted Text:\n", text)


Extracted Text:
 Nutrition Facts

4 servings per container
Serving size 1 1/2 cup (208g)

 

Total Fat 49
Saturated Fat 1.5g
Trans Fat 0g
Cholesterol 5mg
Sodium 430mg
Total Carbohydrate 469
Dietary Fiber 7g
Total Sugars 4g
Includes 2g Added Sugars
tig

 

 

Vitamin D 2meg
Calcium 260mg
Iron 6mg
Potassium 240mg

2%
19%
17%
25%

4%


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords


nltk.download("stopwords")

def clean_text(text):

    text = text.lower()
    text = re.sub(r"[^a-zA-Z\s,]", "", text)
    text = text.replace("\n", " ")
    words = text.split(",")
    words = [word.strip() for word in words]
    words = [word for word in words if word not in stopwords.words("english")]
    return words


ingredients = clean_text(text)
print("Cleaned Ingredients:", ingredients)


df = pd.DataFrame({"Ingredient": ingredients})
print("Structured Data:")
print(df)


Cleaned Ingredients: ['nutrition facts   servings per container serving size   cup g     total fat  saturated fat g trans fat g cholesterol mg sodium mg total carbohydrate  dietary fiber g total sugars g includes g added sugars tig        vitamin d meg calcium mg iron mg potassium mg']
Structured Data:
                                          Ingredient
0  nutrition facts   servings per container servi...


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import pytesseract
from PIL import Image
import re
from difflib import SequenceMatcher


health_scores = {
    "sugar": -2,
    "high fructose corn syrup": -3,
    "artificial flavors": -2,
    "preservatives": -2,
    "sodium nitrate": -3,
    "trans fat": -4,
    "msg": -2,
    "aspartame": -3,
}


FUZZY_THRESHOLD = 0.85


def analyze_health(ingredients_text):
    score = 10
    ingredients_text = ingredients_text.lower()


    match = re.search(r"ingredients\s*[:\-]?\s*(.*)", ingredients_text)
    if match:
        ingredients_line = match.group(1)
    else:
        ingredients_line = ingredients_text


    ingredients = [i.strip() for i in ingredients_line.split(',')]


    for ingredient in ingredients:
        for bad_ingredient, penalty in health_scores.items():
            similarity = SequenceMatcher(None, ingredient, bad_ingredient).ratio()
            if similarity >= FUZZY_THRESHOLD:
                score += penalty
                break

    return max(1, score)

# ===== Main Process =====
image_path = image
image_1 = Image.open(image_path)
extracted_text = pytesseract.image_to_string(image_1)


health_score = analyze_health(extracted_text)
print(f"Health Score: {health_score}/10")



Health Score: 10/10


In [ ]:
!pip install transformers torch datasets pandas nltk


In [ ]:
import torch
from transformers import pipeline


classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased")

def analyze_ingredients_with_bert(ingredients):
    """Use DistilBERT to analyze food ingredient health impact."""
    health_ratings = {}

    for ingredient in ingredients:
        result = classifier(ingredient)[0]
        score = result["score"]


        health_score = round(score * 10)

        health_ratings[ingredient] = health_score

    return health_ratings


ingredients = ["sugar", "high fructose corn syrup", "artificial flavors", "olive oil", "vitamin C"]
bert_analysis = analyze_ingredients_with_bert(ingredients)
print("\nBERT-Based Analysis:\n", bert_analysis)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu



BERT-Based Analysis:
 {'sugar': 5, 'high fructose corn syrup': 5, 'artificial flavors': 5, 'olive oil': 6, 'vitamin C': 5}


In [ ]:
def analyze_health(ingredients_text):
    score = 10

    if isinstance(ingredients_text, list):
        ingredients_text = ','.join(ingredients_text).lower()
    else:
        ingredients_text = ingredients_text.lower()
    for keyword, penalty in health_scores.items():
        if keyword in ingredients_text:
            score += penalty
    return max(1, score)

In [ ]:
image_path = image


text = extract_text(image_path)


ingredients = clean_text(text)


health_score = analyze_health(ingredients)


bert_analysis = analyze_ingredients_with_bert(ingredients)

print("Final Analysis:")
print("Extracted Ingredients:", ingredients)
print(f"Health Score (Rule-Based): {health_score}/10")
print("BERT-Based Health Ratings:", bert_analysis)

Final Analysis:
Extracted Ingredients: ['nutrition facts   servings per container serving size   cup g     total fat  saturated fat g trans fat g cholesterol mg sodium mg total carbohydrate  dietary fiber g total sugars g includes g added sugars tig        vitamin d meg calcium mg iron mg potassium mg']
Health Score (Rule-Based): 4/10
BERT-Based Health Ratings: {'nutrition facts   servings per container serving size   cup g     total fat  saturated fat g trans fat g cholesterol mg sodium mg total carbohydrate  dietary fiber g total sugars g includes g added sugars tig        vitamin d meg calcium mg iron mg potassium mg': 5}
